In [1]:
from IPython.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
from desdeo_emo.EAs import RVEA, NSGAIII, IBEA
from desdeo_problem.testproblems.TestProblems import test_problem_builder
from desdeo_problem import DataProblem
#from desdeo_problem.testproblems.DBMOPP import DBMOPP
from desdeo_tools.utilities import fast_non_dominated_sort, hypervolume_indicator

import matplotlib.pyplot as plt
import sklearn
from pyDOE import lhs
import pandas as pd

In [4]:
#pip install pyDOE

In [5]:
#np.sqrt(mean_squared_error(self.y, y_pred))

from sklearn.metrics import mean_squared_error, r2_score

#r2_score(self.y, y_pred)

## Assignment 2

Generate 100 samples (LHS sampling) with the welded beam design problem (K=2, n=4). 

Build two
types of surrogate models for each objective and ﬁnd the R-squared and RMSE for K fold cross
validations. 

Suggest which models to use (for each objective) for performing optimization based on
the accuracies. 

Finally generate 50 more LHS samples and test the models and show their RMSEs


- Use DESDEO, sklearn, GPy
- Optimize the hyperparameters for Kriging
- You can also try to optimize the hyperparameters for random forest

##### what should be done

100 pistettä jakaa Kfold CVllä.. GPR ja Randomforest obj1 ja obj2. Seurata näiden opetus R2 ja RMSE. Sitten optimoida parameterejä kunnes tyytyväinen ja generoida 50 pistettä lisää, joille sama homma. Testata malleja.


TODO: 
 optimize krigin params.. 
 f1 näyttää niin hyvältä että sitä ei varmaan tarvitse optimoida pikemminkin vain jos löytää miksi niin hyvä
 f2 sitä voisi sitten koittaa parantaa. Luulisi että normalisointi auttaiis mutta en tiedä..
 Onko probleema edes formuloitu oikein kun ei ole og paperia missä biobjective formulointi löytynyt?

## Result 

|  |  | Objective 1 K | | Objective 1 RF |  Objective 2 K | Objective 2 RF |
| --- | --- | --- | --- | --- | -- | -- |
| Kfold CV R^2 |  |  |   |   |   | |
| kfold CV RMSE |  |  |  |  |   | |
| Test R^2 |  | |  |   |  | |
| Test RMSE |  |  |  |   |   | |

In [6]:
# the problem, should be correct one

P = 6000
C = 0.000036578 # check if enough zeroes should be 3.6578 times 10^-4

def f1(x):
    return 1.10471*x[:,0]**2*x[:,1] + 0.04811*x[:,2]*x[:,3]*(14 + x[:,1])


def f2(x):
    return (P / (x[:,3]*x[:,2]**3)) * C


The two objectives are the fabrication cost of the beam and the deflection of the end of the beam under the applied load P. The load P is fixed at 6,000 lbs, and the distance L is fixed at 14 in.

The design variables are:

x(1) = h, the thickness of the welds

x(2) = l, the length of the welds

x(3) = t, the height of the beam

x(4) = b, the width of the beam

Krigin and Random Forests were suggested.. 

In [7]:
def create_samples(samples):
    # create samples 
    x = lhs(4, samples)
    # could rescale if needed. sklearn GPR assumes that mean is  0 ?
    #x = (x - 0.5)
    return x

### GRP from sklearn 
- TODO : kfold cross validation 

In [8]:
from sklearn.pipeline import Pipeline

from sklearn.gaussian_process.kernels import RBF, ExpSineSquared, WhiteKernel, RationalQuadratic, DotProduct, ConstantKernel, Matern

In [9]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

## CREATE SAMPLES WITH LHS

it really should just be as simple as below ?!
https://scikit-learn.org/stable/modules/cross_validation.html

In [49]:
x = create_samples(100)
func1 = f1(x)
func2 = f2(x)
f = np.vstack((func1, func2)).T

data = pd.DataFrame(np.hstack((x, f)), columns=["h", "l", "t", "b", "f1", "f2"])
f1_mean = func1.mean()
f2_mean = func2.mean()
print(f"Means of objective values with samples f1: {f1_mean} and f2: {f2_mean}")
data[:5]

Means of objective values with samples f1: 0.3349113620411198 and f2: 11301.56268934375


,h,l,t,b,f1,f2
0,0.393126,0.599178,0.547354,0.342836,0.234099,3.903735
1,0.346647,0.812066,0.553611,0.299163,0.225821,4.323646
2,0.988790,0.031050,0.191425,0.492253,0.097144,63.560791
3,0.810163,0.554175,0.959442,0.764266,0.915263,0.325140
4,0.589279,0.621323,0.927859,0.607913,0.635122,0.451943


In [51]:
from sklearn.preprocessing import MinMaxScaler

# as assumed works with train but if new samples wont get scaled worse results
#sc = MinMaxScaler()
#f = sc.fit_transform(f)

#### from the data 
as one can see x ranges from [0,1] bc LHS. F1 seems to get values with mean of about 0.35 and f2 with way bigger numbers. So should one normalize f2 separetely or to do what with this big difference ?

In [52]:
from sklearn.gaussian_process import kernels
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.gaussian_process import GaussianProcessRegressor as GPR

from sklearn.model_selection import train_test_split

x, x_test, f, y_test = train_test_split(x, f, test_size=0.20, random_state=7)

func1 = f[:,0] # train ys
func2 = f[:,1]

y1 = y_test[:,0] # test ys
y2 = y_test[:,1]

seed = 7

parameters = {
    "alpha": [1e-10, 1e-8, 1e-6, 1e-4,
                1e-2, 1e-0, 1e2, 1e4],
    "kernel": [
        kernels.RBF(length_scale_bounds=(1e-10, 1000000)),
        kernels.RationalQuadratic(length_scale_bounds=(1e-10, 1000000)),
        kernels.DotProduct(sigma_0_bounds=(1e-10, 1000000)),
        kernels.Matern(length_scale_bounds=(1e-10, 1000000)),
        kernels.ExpSineSquared(length_scale_bounds=(1e-10, 1000000))
    ]
}

# GridSearchCV does crossvalidation
gpr = GridSearchCV(GPR(normalize_y=True, n_restarts_optimizer=1,random_state=seed), parameters, cv=10)

gpr2 = GridSearchCV(GPR(normalize_y=True,n_restarts_optimizer=1,random_state=seed), parameters, cv=10)

# fit best parameters
gpr.fit(x, func1)
gpr = gpr.best_estimator_
print(gpr)

gpr2.fit(x, func2)
gpr2 = gpr2.best_estimator_
print(gpr2)

GaussianProcessRegressor(alpha=1e-06, kernel=RBF(length_scale=1),
                         n_restarts_optimizer=1, normalize_y=True,
                         random_state=7)
GaussianProcessRegressor(alpha=0.01,
                         kernel=RationalQuadratic(alpha=1, length_scale=1),
                         n_restarts_optimizer=1, normalize_y=True,
                         random_state=7)


In [53]:
m, std = gpr.predict(x_test, return_std=True)
print(mean_squared_error(m,y1, squared=False)) # dunno if this correct ?
print(r2_score(m, y1))

0.008538916592643594
0.9983027103242501


In [54]:
m2, std = gpr2.predict(x_test, return_std=True)
print(mean_squared_error(m2, y2, squared=False))
print(r2_score(m2, y2))

0.013301647640069716
-1.852698647271667


## F1 with GPR

In [231]:


gpr = GPR()
gpr.fit(x,func1)

# this is jsut using cross val score. But need to do the splitting? 
scores = cross_val_score(gpr, x, func1, cv=5, scoring='r2')
print(scores)

m, std = gpr.predict(x, return_std=True)
print(mean_squared_error(m,func1, squared=False)) # dunno if this correct ?
print(r2_score(mean, func1))

[0.9996066  0.99985174 0.99895904 0.99975568 0.99992012]
7.926660062146229e-08


## F1 with RF

In [232]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(random_state=7)
regr.fit(x, func1)
scores = cross_val_score(regr, x, func1, cv=5, scoring='r2')
print(scores)
rm = regr.predict(x)
print(mean_squared_error(rm,func1, squared=False))
#print(r2_score(rm, func1))

[0.9304012  0.87473371 0.74688544 0.86391013 0.92384354]
0.03684535544060995


## F2 with GPR

In [233]:
gpr2 = GPR()
#gpr2 = GPR(normalize_y=True)
gpr2.fit(x,func2)

# this is jsut using cross val score. But need to do the splitting? 
scores2 = cross_val_score(gpr2, x, func2, cv=5, scoring='r2')
print(scores2)

m2, std = gpr2.predict(x, return_std=True)
print(mean_squared_error(m2,func2, squared=False)) # dunno if this correct ?

[-1.56805751e+10 -1.16227983e+05 -4.03055748e+05 -5.39012560e-02
 -2.57163880e+07]
53.20269192003474


## F2 with RF

In [234]:
regr2 = RandomForestRegressor(random_state=7)
regr2.fit(x, func2)

rscores2 = cross_val_score(regr2, x, func2, cv=5, scoring='r2')
print(rscores2)

rm2 = regr2.predict(x)
print(mean_squared_error(rm2,func2, squared=False))
#print(r2_score(rm2, func2))

[-1.54371871e+00 -4.88578321e+03 -1.32843025e+03 -4.84898502e-02
  1.62984707e-01]
869825.7256647566


## Get test samples

In [55]:
# gen the test data

x_test = create_samples(50)
func_test1 = f1(x_test)
func_test2 = f2(x_test)
f1t_mean = func_test1.mean()
f2t_mean = func_test2.mean()
print(f"Means of test objective values with test samples f1: {f1t_mean} and f2: {f2t_mean}")
f_test = np.vstack((func_test1, func_test2)).T
test_data = pd.DataFrame(np.hstack((x_test, f_test)), columns=["h", "l", "t", "b", "f1", "f2"])
test_data[:5]

Means of test objective values with test samples f1: 0.36090846981946767 and f2: 4432.3807765978145


,h,l,t,b,f1,f2
0,0.349439,0.398685,0.503116,0.718311,0.304125,2.399137
1,0.218137,0.514260,0.986138,0.031180,0.048503,7.339804
2,0.251976,0.102797,0.632866,0.499367,0.221634,1.733868
3,0.795385,0.253852,0.611071,0.184814,0.254858,5.204284
4,0.185183,0.121076,0.148283,0.883903,0.093630,76.154294


In [56]:
mean, std = gpr.predict(x_test, return_std=True)
print(mean_squared_error(mean,func_test1, squared=False)) # dunno if this correct ?
print(r2_score(mean, func_test1))

0.11092284842847702
0.6845228180803303


In [57]:
mean, std = gpr2.predict(x_test, return_std=True)
print(mean_squared_error(mean,func_test2, squared=False)) # dunno if this correct ?
print(r2_score(mean, func_test2))

27570.000925251625
-520315775869.29626


## GPR F1 TEST

In [236]:
mean, std = gpr.predict(x_test, return_std=True)
print(mean_squared_error(mean,func_test1, squared=False)) # dunno if this correct ?
print(r2_score(mean, func_test1))

0.0028721956277529335
0.999881178980473


## RF F1 TEST

In [237]:
rmean = regr.predict(x_test)
print(mean_squared_error(rmean,func_test1, squared=False)) # dunno if this correct ?
print(r2_score(rmean, func_test1))

0.0896142328508594
0.8131621493260025


## GPR F2 TEST

In [238]:
mean2, std2 = gpr2.predict(x_test,return_std=True)
print(mean_squared_error(mean2,func_test2, squared=False))
print(r2_score(mean2, func_test2))

5667383.045929776
-0.001482976046963902


## RF F2 TEST

In [239]:
rmean2 = regr2.predict(x_test)
print(mean_squared_error(rmean2,func_test2, squared=False))
print(r2_score(rmean2, func_test2))

466451.9728315159
0.15984132791606342


## TODO:

to me the above looks correctly done BUT gpr1 seems to be too good and gpr2 seems to be too bad?

if f2 has smaller mean that test_f2, our r2 goes negative. DUnno what to do with this


Atleast random forests are consistent with the issue. Even though wont be as good for f1, still bad for f2 and in the same way.

RF seems to work better f2, especially if train and test set has similar magnitude mean. This sounds like I should someway properly do the normalization.

In [84]:

## sklearn, with func1
#gpr = GPR()
#gpr.fit(x,func1)
#mean, std = gpr.predict(x, return_std=True)
#print(mean_squared_error(mean,func1)) # dunno if this correct ?
#print(r2_score(mean, func1))

In [86]:
## sklearn, with func2
#gpr = GPR()
#gpr.fit(x,func2)
#mean, std = gpr.predict(x, return_std=True)
#print(mean_squared_error(mean,func2))
#print(r2_score(mean, func2))

### RF from sklearn 
- TODO : kfold cross validation 

In [26]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(random_state=7)
regr.fit(x, func1)
m = regr.predict(x)
print(mean_squared_error(m,func1))
print(r2_score(m, func1))

0.0015696025359983495
0.9764521506416795


In [25]:
regr = RandomForestRegressor(random_state=7)
regr.fit(x, func2)
m = regr.predict(x)
print(mean_squared_error(m,func2))
print(r2_score(m, func2))

42353013631.449585
0.5427258801350856


 ## trying with desdeo

In [92]:
problem = DataProblem(data=data, variable_names=["h", "l", "t", "b"], objective_names=["f1", "f2"])

In [93]:
from desdeo_problem.surrogatemodels.SurrogateModels import GaussianProcessRegressor

In [57]:
#
# Desdeo styl
y = problem.train(gpr)
#problem.train(GaussianProcessRegressor)
#gpr = GPR()

y = problem.train(GPR)
y
#gpr.score(x,y)